#  Problem Statement : Next Word Prediction

## Statistical language model


![](LanguageModel.PNG)

In [ ]:
##CHALLENGES

#Sensitivity to the training corpus
#Smoothing

In [ ]:
#This assumption that the probability of a word depends only on the previous word is also known as Markov assumption.



# Import Required Libraries:

In [1]:
import pandas as pd
import numpy as np

import pickle
import numpy as np
import os


import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Embedding, LSTM, Dense



#john started a textile factory

# transformer models -- attention model

#BERT
#GPT-1/2/3 -- OPEN AI





In [2]:
"""
http://www.gutenberg.org/cache/epub/5200/pg5200.txt

"""


file = open("metamorphosis.txt", "r", encoding = "utf8")

lines = []

for i in file:
    lines.append(i)
    
    

The First Line:  ﻿One morning, when Gregor Samsa woke from troubled dreams, he found

The Last Line:  first to get up and stretch out her young body.


In [55]:

print("First Line: ", lines[0])

print("Last Line: ", lines[-1])



First Line:  ﻿One morning, when Gregor Samsa woke from troubled dreams, he found

Last Line:  first to get up and stretch out her young body.


lines=['\ufeffOne morning, when Gregor Samsa woke from troubled dreams, he found\n',
 'himself transformed in his bed into a horrible vermin.  He lay on\n',
 'his armour-like back, and if he lifted his head a little he could\n',
 'see his brown belly, slightly domed and divided by arches into stiff\n',
 'sections.  The bedding was hardly able to cover it and seemed ready\n',
 'to slide off any moment.  His many legs, pitifully thin compared\n',
 'with the size of the rest of him, waved about helplessly as he\n',
 'looked.\n',
 '\n',
 '"What\'s happened to me?" he thought.  It wasn\'t a dream.  His room,\n',
 'a proper human room although a little too small, lay peacefully\n',
 'between its four familiar walls.  A collection of textile samples\n',
 'lay spread out on the table - Samsa was a travelling salesman - and\n',
 'above it there hung a picture that he had recently cut out of an\n',
 'illustrated magazine and housed in a nice, gilded frame.  It showed\n',
 'a lady fitted out with a fur hat and fur boa who sat upright,\n',
 'raising a heavy fur muff that covered the whole of her lower arm\n']


In [22]:
lines



['\ufeffOne morning, when Gregor Samsa woke from troubled dreams, he found\n',
 'himself transformed in his bed into a horrible vermin.  He lay on\n',
 'his armour-like back, and if he lifted his head a little he could\n',
 'see his brown belly, slightly domed and divided by arches into stiff\n',
 'sections.  The bedding was hardly able to cover it and seemed ready\n',
 'to slide off any moment.  His many legs, pitifully thin compared\n',
 'with the size of the rest of him, waved about helplessly as he\n',
 'looked.\n',
 '\n',
 '"What\'s happened to me?" he thought.  It wasn\'t a dream.  His room,\n',
 'a proper human room although a little too small, lay peacefully\n',
 'between its four familiar walls.  A collection of textile samples\n',
 'lay spread out on the table - Samsa was a travelling salesman - and\n',
 'above it there hung a picture that he had recently cut out of an\n',
 'illustrated magazine and housed in a nice, gilded frame.  It showed\n',
 'a lady fitted out with a fur

# Clean the data:

In [56]:
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')

data[:300]

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.  He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections.  The bedding was hardl'

In [57]:
import string

translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation)) #map punctuation to space
#ValueError: the first two maketrans arguments must have equal length


new_data = data.translate(translator)

new_data[:1000]



'One morning  when Gregor Samsa woke from troubled dreams  he found himself transformed in his bed into a horrible vermin   He lay on his armour like back  and if he lifted his head a little he could see his brown belly  slightly domed and divided by arches into stiff sections   The bedding was hardly able to cover it and seemed ready to slide off any moment   His many legs  pitifully thin compared with the size of the rest of him  waved about helplessly as he looked    What s happened to me   he thought   It wasn t a dream   His room  a proper human room although a little too small  lay peacefully between its four familiar walls   A collection of textile samples lay spread out on the table   Samsa was a travelling salesman   and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice  gilded frame   It showed a lady fitted out with a fur hat and fur boa who sat upright  raising a heavy fur muff that covered the whole of her lower arm 

In [75]:
len(new_data)

119163

In [21]:
string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [58]:
z = []

for i in data.split():
    if i not in z:
        z.append(i)
        
data = ' '.join(z)

data[:1000]

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on armour-like back, and if lifted head little could see brown belly, slightly domed divided by arches stiff sections. The bedding was hardly able to cover it seemed ready slide off any moment. His many legs, pitifully thin compared with the size of rest him, waved about helplessly as looked. "What\'s happened me?" thought. It wasn\'t dream. room, proper human room although too small, peacefully between its four familiar walls. A collection textile samples spread out table - travelling salesman above there hung picture that had recently cut an illustrated magazine housed nice, gilded frame. showed lady fitted fur hat boa who sat upright, raising heavy muff covered whole her lower arm towards viewer. then turned look window at dull weather. Drops rain be heard hitting pane, which made him feel quite sad. "How I sleep bit longer forget all this nonsense", thou

In [76]:
len(data)

29361

### Tokenization:

In [23]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts([data])


In [54]:

show_words = tokenizer.texts_to_sequences([data])[0]
show_words



[17,
 53,
 293,
 2,
 18,
 729,
 135,
 730,
 294,
 8,
 731,
 19,
 732,
 9,
 295,
 54,
 733,
 296,
 297,
 734,
 8,
 735,
 29,
 736,
 30,
 55,
 20,
 56,
 737,
 136,
 57,
 298,
 137,
 738,
 138,
 299,
 739,
 740,
 300,
 741,
 301,
 742,
 58,
 743,
 31,
 302,
 744,
 59,
 745,
 4,
 303,
 304,
 746,
 305,
 747,
 60,
 295,
 748,
 139,
 749,
 750,
 751,
 140,
 58,
 752,
 141,
 306,
 21,
 753,
 142,
 754,
 143,
 144,
 61,
 62,
 63,
 145,
 4,
 755,
 307,
 32,
 756,
 308,
 32,
 146,
 33,
 309,
 310,
 757,
 758,
 759,
 760,
 147,
 296,
 761,
 762,
 763,
 764,
 10,
 64,
 148,
 765,
 766,
 22,
 767,
 311,
 5,
 65,
 768,
 769,
 312,
 770,
 771,
 772,
 149,
 773,
 313,
 774,
 314,
 775,
 315,
 316,
 776,
 150,
 777,
 66,
 778,
 779,
 780,
 781,
 317,
 67,
 782,
 318,
 783,
 784,
 6,
 319,
 68,
 151,
 152,
 785,
 320,
 321,
 786,
 34,
 153,
 787,
 322,
 323,
 788,
 21,
 324,
 789,
 325,
 154,
 326,
 155,
 790,
 156,
 791,
 23,
 11,
 792,
 145,
 35,
 69,
 793,
 70,
 157,
 794,
 795,
 71,
 158,
 159,
 796

In [74]:
data

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on armour-like back, and if lifted head little could see brown belly, slightly domed divided by arches stiff sections. The bedding was hardly able to cover it seemed ready slide off any moment. His many legs, pitifully thin compared with the size of rest him, waved about helplessly as looked. "What\'s happened me?" thought. It wasn\'t dream. room, proper human room although too small, peacefully between its four familiar walls. A collection textile samples spread out table - travelling salesman above there hung picture that had recently cut an illustrated magazine housed nice, gilded frame. showed lady fitted fur hat boa who sat upright, raising heavy muff covered whole her lower arm towards viewer. then turned look window at dull weather. Drops rain be heard hitting pane, which made him feel quite sad. "How I sleep bit longer forget all this nonsense", thou

In [53]:
tokenizer.word_index.items()





dict_items([('now', 1), ('gregor', 2), ('well', 3), ('it', 4), ('that', 5), ('then', 6), ('father', 7), ('he', 8), ('in', 9), ('out', 10), ('this', 11), ('so', 12), ('before', 13), ('no', 14), ('mother', 15), ('grete', 16), ('one', 17), ('samsa', 18), ('himself', 19), ('and', 20), ('him', 21), ('there', 22), ('all', 23), ('you', 24), ('yes', 25), ('again', 26), ('work', 27), ('here', 28), ('on', 29), ('like', 30), ('was', 31), ('room', 32), ('too', 33), ('be', 34), ('but', 35), ('god', 36), ('is', 37), ('soon', 38), ('enough', 39), ('would', 40), ('seven', 41), ('did', 42), ('come', 43), ('round', 44), ('door', 45), ('while', 46), ('said', 47), ('already', 48), ('will', 49), ('help', 50), ('we', 51), ('anyway', 52), ('morning', 53), ('bed', 54), ('back', 55), ('if', 56), ('little', 57), ('the', 58), ('to', 59), ('moment', 60), ("what's", 61), ('happened', 62), ('me', 63), ('table', 64), ('had', 65), ('upright', 66), ('her', 67), ('look', 68), ('something', 69), ('do', 70), ('right', 71

In [27]:

tokenizer.fit_on_texts([data])


sequence_data = tokenizer.texts_to_sequences([data])[0]

sequence_data[:10]


#[we, visit, agra, we, liked, agra]
#[1 33 44 1 23 44]


#WARM STATE=true
#TRANSFER LEARNING

[17,
 53,
 293,
 2,
 18,
 729,
 135,
 730,
 294,
 8,
 731,
 19,
 732,
 9,
 295,
 54,
 733,
 296,
 297,
 734,
 8,
 735,
 29,
 736,
 30,
 55,
 20,
 56,
 737,
 136,
 57,
 298,
 137,
 738,
 138,
 299,
 739,
 740,
 300,
 741,
 301,
 742,
 58,
 743,
 31,
 302,
 744,
 59,
 745,
 4,
 303,
 304,
 746,
 305,
 747,
 60,
 295,
 748,
 139,
 749,
 750,
 751,
 140,
 58,
 752,
 141,
 306,
 21,
 753,
 142,
 754,
 143,
 144,
 61,
 62,
 63,
 145,
 4,
 755,
 307,
 32,
 756,
 308,
 32,
 146,
 33,
 309,
 310,
 757,
 758,
 759,
 760,
 147,
 296,
 761,
 762,
 763,
 764,
 10,
 64,
 148,
 765,
 766,
 22,
 767,
 311,
 5,
 65,
 768,
 769,
 312,
 770,
 771,
 772,
 149,
 773,
 313,
 774,
 314,
 775,
 315,
 316,
 776,
 150,
 777,
 66,
 778,
 779,
 780,
 781,
 317,
 67,
 782,
 318,
 783,
 784,
 6,
 319,
 68,
 151,
 152,
 785,
 320,
 321,
 786,
 34,
 153,
 787,
 322,
 323,
 788,
 21,
 324,
 789,
 325,
 154,
 326,
 155,
 790,
 156,
 791,
 23,
 11,
 792,
 145,
 35,
 69,
 793,
 70,
 157,
 794,
 795,
 71,
 158,
 159,
 796

In [31]:
len(sequence_data)

3890

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on armour-like back, and if lifted head little could see brown belly, slightly domed divided by arches stiff sections. The bedding was hardly able to cover it seemed ready slide off any moment. His many legs, pitifully thin compared with the size of rest him, waved about helplessly as looked. "What\'s happened me?" thought. It wasn\'t dream. room, proper human room although too small, peacefully between its four familiar walls. A collection textile samples spread out table - travelling salesman above there hung picture that had recently cut an illustrated magazine housed nice, gilded frame. showed lady fitted fur hat boa who sat upright, raising heavy muff covered whole her lower arm towards viewer. then turned look window at dull weather. Drops rain be heard hitting pane, which made him feel quite sad. "How I sleep bit longer forget all this nonsense", thou

In [32]:
vocab_size = len(tokenizer.word_index) + 1

print(vocab_size)


#bec index starts with 0 upto  --  2616

2617


In [35]:
sequence_data[0:10]


#1920 --->>> actual word



[17, 53, 293, 2, 18, 729, 135, 730, 294, 8]

In [8]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))

sequences = np.array(sequences)

sequences[:10]


#[17, 53, 293, 2, 18, 729, 135, 730, 294, 8]
#preparing I/O format

The Length of sequences are:  3889


array([[ 17,  53],
       [ 53, 293],
       [293,   2],
       [  2,  18],
       [ 18, 729],
       [729, 135],
       [135, 730],
       [730, 294],
       [294,   8],
       [  8, 731]])

In [65]:
X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)

y = np.array(y)

print(X, y)


#n-grams--unigrams--bigrams---trigrams
#[  [17 53]   [53 293]  [293 2]]


###############



[  17   53  293 ... 2615  294  591] [  53  293    2 ...  294  591 2616]


In [66]:
y.shape

(3889,)

In [68]:
print("The Data is: ", X[:50])

print("The responses are: ", y[:50])


The Data is:  [ 17  53 293   2  18 729 135 730 294   8 731  19 732   9 295  54 733 296
 297 734   8 735  29 736  30  55  20  56 737 136  57 298 137 738 138 299
 739 740 300 741 301 742  58 743  31 302 744  59 745   4]
The responses are:  [ 53 293   2  18 729 135 730 294   8 731  19 732   9 295  54 733 296 297
 734   8 735  29 736  30  55  20  56 737 136  57 298 137 738 138 299 739
 740 300 741 301 742  58 743  31 302 744  59 745   4 303]


In [45]:
vocab_size



2617

In [71]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y, num_classes=vocab_size)

y[:50]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [72]:
y.shape

(3889, 2617)

In [ ]:

#IndexError: index 53 is out of bounds for axis 1 with size 22



y[:5]  #shape (10177513, 2617)  -- 99.2 GiB of memory u need to display this

#IMPORTANT:::
#MemoryError: Unable to allocate 99.2 GiB for an array with shape (10177513, 2617) and data type float32


### Creating the Keras tf Model:

In [13]:

model = Sequential()

model.add(Embedding(vocab_size, 10, input_length=1))

model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))

model.add(Dense(1000, activation="relu"))

model.add(Dense(vocab_size, activation="softmax"))



In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             26170     
_________________________________________________________________
lstm (LSTM)                  (None, 1, 1000)           4044000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense (Dense)                (None, 1000)              1001000   
_________________________________________________________________
dense_1 (Dense)              (None, 2617)              2619617   
Total params: 15,694,787
Trainable params: 15,694,787
Non-trainable params: 0
_________________________________________________________________


### Plot The Model:

In [ ]:
#You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


from tensorflow import keras

from keras.utils.vis_utils import plot_model

keras.utils.plot_model(model, to_file='model.png', show_layer_names=True)

### Callbacks:

In [17]:
#OUTPUT FILES FOR FUTURE PREDICTIONS


from tensorflow.keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')



# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))




from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard


reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

### Compile keras Model with hyperparameters

In [18]:

#lr=learning rate
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))




### Fit keras tf Model:

In [19]:

model.fit(X, y, epochs=150, batch_size=64) 
          #callbacks=[checkpoint, reduce, tensorboard_Visualization])



Epoch 1/150
 1/61 [..............................] - ETA: 0s - loss: 7.8698WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
61/61 [==============================] - ETA: 0s - loss: 7.8754
Epoch 00001: loss improved from inf to 7.87536, saving model to nextword1.h5
61/61 [==============================] - 23s 371ms/step - loss: 7.8754
Epoch 2/150
61/61 [==============================] - ETA: 0s - loss: 7.8619
Epoch 00002: loss improved from 7.87536 to 7.86192, saving model to nextword1.h5
61/61 [==============================] - 18s 298ms/step - loss: 7.8619
Epoch 3/150
61/61 [==============================] - ETA: 0s - loss: 7.8126
Epoch 00003: loss improved from 7.86192 to 7.81256, saving model to nextword1.h5
61/61 [==============================] - 19s 3

61/61 [==============================] - ETA: 0s - loss: 1.8200
Epoch 00070: loss improved from 1.84339 to 1.81996, saving model to nextword1.h5
61/61 [==============================] - 19s 305ms/step - loss: 1.8200
Epoch 71/150
61/61 [==============================] - ETA: 0s - loss: 1.7901
Epoch 00071: loss improved from 1.81996 to 1.79013, saving model to nextword1.h5
61/61 [==============================] - 18s 300ms/step - loss: 1.7901
Epoch 72/150
61/61 [==============================] - ETA: 0s - loss: 1.7981
Epoch 00072: loss did not improve from 1.79013
61/61 [==============================] - 16s 270ms/step - loss: 1.7981
Epoch 73/150
61/61 [==============================] - ETA: 0s - loss: 1.7756
Epoch 00073: loss improved from 1.79013 to 1.77555, saving model to nextword1.h5
61/61 [==============================] - 22s 366ms/step - loss: 1.7756
Epoch 74/150
61/61 [==============================] - ETA: 0s - loss: 1.7847
Epoch 00074: loss did not improve from 1.77555
61/61 [

Epoch 144/150
61/61 [==============================] - ETA: 0s - loss: 0.6803
Epoch 00144: loss improved from 0.68090 to 0.68034, saving model to nextword1.h5
61/61 [==============================] - 21s 346ms/step - loss: 0.6803
Epoch 145/150
61/61 [==============================] - ETA: 0s - loss: 0.6794
Epoch 00145: loss improved from 0.68034 to 0.67936, saving model to nextword1.h5
61/61 [==============================] - 19s 304ms/step - loss: 0.6794
Epoch 146/150
61/61 [==============================] - ETA: 0s - loss: 0.6764
Epoch 00146: loss improved from 0.67936 to 0.67637, saving model to nextword1.h5
61/61 [==============================] - 18s 289ms/step - loss: 0.6764
Epoch 147/150
61/61 [==============================] - ETA: 0s - loss: 0.6785
Epoch 00147: loss did not improve from 0.67637
61/61 [==============================] - 16s 268ms/step - loss: 0.6785
Epoch 148/150
61/61 [==============================] - ETA: 0s - loss: 0.6776
Epoch 00148: loss did not improve fr

# now we are ready for prediction using the next notebook

## Neural Networks - RNN-LSTM - Language Model - prediction.ipynb